In [1]:
import numpy as np
import pandas as pd
from parallel_pandas import ParallelPandas
from rdkit import Chem

In [2]:
from curation.utils import *
from curation.cleaning import *

In [3]:
ParallelPandas.initialize(n_cpu=16, split_factor=4, disable_pr_bar=True)

In [4]:
smiles = pd.read_csv('data/cyp_train.csv')
smiles = pd.Series(smiles.iloc[:, 1])
smiles

0        CN(C(=O)Cc1ccccc1)[C@@H]1CC[C@@]2(CCCO2)C[C@H]...
1             COc1ccc(/C=N/NC(N)=S)cc1OC(=O)c1cccc2ccccc12
2                        COc1ccccc1COC(=O)Nc1c(C)nn(C)c1Cl
3        COc1ccc(Cl)cc1S(=O)(=O)N1CCN(C(=O)c2cc(OC)c(OC...
4                   O=C(c1ccccc1F)c1cn(Cc2ccccc2)c2ccccc12
                               ...                        
41230                  NS(=O)(=O)c1cc2c(cc1Cl)NC=NS2(=O)=O
41231                        CN1CCCC1CCNC1C2CC3CC(C2)CC1C3
41232        Cn1c(=O)c(-c2ccc(F)cc2)nc2cnc(OCc3ccccc3)nc21
41233                 O=C(CNS(=O)(=O)c1ccc(Br)cc1)N1CCOCC1
41234    CCc1c(O)c2c(c(O)c1C(C)=O)C(=O)c1c(cc(O)c(C(=O)...
Name: compound, Length: 41235, dtype: object

In [5]:
smiles = smiles.p_apply(smiles_validation)
smiles

0        CN(C(=O)Cc1ccccc1)[C@@H]1CC[C@@]2(CCCO2)C[C@H]...
1             COc1ccc(/C=N/NC(N)=S)cc1OC(=O)c1cccc2ccccc12
2                        COc1ccccc1COC(=O)Nc1c(C)nn(C)c1Cl
3        COc1ccc(Cl)cc1S(=O)(=O)N1CCN(C(=O)c2cc(OC)c(OC...
4                   O=C(c1ccccc1F)c1cn(Cc2ccccc2)c2ccccc12
                               ...                        
41230                  NS(=O)(=O)c1cc2c(cc1Cl)NC=NS2(=O)=O
41231                        CN1CCCC1CCNC1C2CC3CC(C2)CC1C3
41232        Cn1c(=O)c(-c2ccc(F)cc2)nc2cnc(OCc3ccccc3)nc21
41233                 O=C(CNS(=O)(=O)c1ccc(Br)cc1)N1CCOCC1
41234    CCc1c(O)c2c(c(O)c1C(C)=O)C(=O)c1c(cc(O)c(C(=O)...
Name: compound, Length: 41235, dtype: object

In [6]:
numbers = count_for_failed_smiles(smiles)
numbers

0

In [7]:
smiles = smiles.p_apply(remove_mixtures)
smiles

0        CN(C(=O)Cc1ccccc1)[C@@H]1CC[C@@]2(CCCO2)C[C@H]...
1             COc1ccc(/C=N/NC(N)=S)cc1OC(=O)c1cccc2ccccc12
2                        COc1ccccc1COC(=O)Nc1c(C)nn(C)c1Cl
3        COc1ccc(Cl)cc1S(=O)(=O)N1CCN(C(=O)c2cc(OC)c(OC...
4                   O=C(c1ccccc1F)c1cn(Cc2ccccc2)c2ccccc12
                               ...                        
41230                  NS(=O)(=O)c1cc2c(cc1Cl)NC=NS2(=O)=O
41231                        CN1CCCC1CCNC1C2CC3CC(C2)CC1C3
41232        Cn1c(=O)c(-c2ccc(F)cc2)nc2cnc(OCc3ccccc3)nc21
41233                 O=C(CNS(=O)(=O)c1ccc(Br)cc1)N1CCOCC1
41234    CCc1c(O)c2c(c(O)c1C(C)=O)C(=O)c1c(cc(O)c(C(=O)...
Name: compound, Length: 41235, dtype: object

In [10]:
numbers = count_for_failed_smiles(smiles)
numbers

11

In [9]:
smiles = smiles.p_apply(remove_inorganic_compound)
smiles

0        CN(C(=O)Cc1ccccc1)[C@@H]1CC[C@@]2(CCCO2)C[C@H]...
1             COc1ccc(/C=N/NC(N)=S)cc1OC(=O)c1cccc2ccccc12
2                        COc1ccccc1COC(=O)Nc1c(C)nn(C)c1Cl
3        COc1ccc(Cl)cc1S(=O)(=O)N1CCN(C(=O)c2cc(OC)c(OC...
4                   O=C(c1ccccc1F)c1cn(Cc2ccccc2)c2ccccc12
                               ...                        
41230                  NS(=O)(=O)c1cc2c(cc1Cl)NC=NS2(=O)=O
41231                        CN1CCCC1CCNC1C2CC3CC(C2)CC1C3
41232        Cn1c(=O)c(-c2ccc(F)cc2)nc2cnc(OCc3ccccc3)nc21
41233                 O=C(CNS(=O)(=O)c1ccc(Br)cc1)N1CCOCC1
41234    CCc1c(O)c2c(c(O)c1C(C)=O)C(=O)c1c(cc(O)c(C(=O)...
Name: compound, Length: 41235, dtype: object